In [1]:
from enhanced_lib.exchange import ExchangeCache,Account

account = Account(
    'http://localhost:8000',
    "main_account",
    "sub_account",
)

exchange = ExchangeCache(account, "BTCUSDT")

# await exchange.fetch_latest(False)

# exchange.open_orders.get_tp_orders('long',True)

# await account.fetch_config('BTCUSDT')
await exchange.fetch_latest(True)
# exchange.get_next_tradable_zone("long")
exchange.get_next_tradable_zone("short")
# exchange.get_zone_for_position('long')
# exchange.maximum_sell_size
# exchange.future_instance.config.get_trading_zones('kind')
# exchange.open_orders.get_sl_orders('short',True)
# await account.update_config_fields('BTCUSDT',{'budget':400,'max_size':0.21})
# await account.update_config_resistance('BTCUSDT')
# account.general_config['config']['BTCUSDT']
# exchange.future_instance.config.as_dict()
future_instance = exchange.future_instance

# future_instance.get_trading_zones('long')
# future_instance.config.long_liquidation_price
# future_instance.config.as_dict()
# zones = future_instance.config.get_trading_zones('short')
# zones
# stop = future_instance.config.long_liquidation_price
# stop
# app_config = future_instance.config.app_config
# app_config.entry = 44779.3
# app_config.stop = stop
# # app_config.raw = True
# # workers.determine_optimum_reward(app_config)
# exchange.maximum_sell_size
# exchange.get_zone_for_position('long')
# exchange.get_zone_for_position('long')
# exchange.get_next_tradable_zone('long')
# exchange.config_params_for_future_trades2('long',True)

In [1]:
from enhanced_lib.calculations.shared import build_config
from enhanced_lib.calculations.workers.optimum_risk_reward import eval_func,determine_optimum_reward,run_in_parallel
from enhanced_lib.calculations.workers.utils import chunks_in_threads,run_in_threads
app_config = future_instance.config.app_config
app_config.entry = 41993.9
app_config.stop = 44779.3
app_config.kind = 'short'
app_config.risk_reward = 80
# app_config.risk_per_trade = 70
print('app_config',app_config)
# app_config.support = 41900
# app_config.resistance  = 45000
# signal = build_config(app_config,{
#     # 'raw_instance':True,
#     'decimal_places':app_config.decimal_places,
#     'increase':True,
#     'risk_reward':80,
#     'risk_per_trade': 6,
#     'entry': 44200,
#     'kind': 'long',
#     'stop': 42000,
#     'currentEntry':44200
# })
# signal
# # signal.get_bulk_trade_zones(44200,kind='long')
app_config.strategy = 'entry'
# eval_func(80,app_config)
# for i in range(30,199,1):
#     uu = eval_func(i,app_config)
#     # print('uu',len(uu['result']))
# func = run_in_parallel(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=8)
# func
# bb = chunks_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=4,num_threads=40)
# len(bb)
# func = run_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],num_threads=2)
# determine_optimum_reward(app_config)
# exchange.future_instance.config.long_liquidation_price
# exchange.get_next_tradable_zone('long')
# exchange.get_next_tradable_zone('short')
exchange.config_params_for_future_trades('short',True,ignore=True)

NameError: name 'future_instance' is not defined